In [ ]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:

# from datasets import load_dataset
import pandas as pd

import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)
from transformers import BitsAndBytesConfig
from peft import PeftModel, PeftConfig

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2025-04-15 17:37:55.120277: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appr

In [3]:

# Your adapter repo or local dir
# peft_model_id = "mika5883/ru_qwen_gec" # or your output_dir path
# peft_model_id = 'mika5883/ru_qwen_gec_Ag_art'
peft_model_id = 'mika5883/ru_qwen7b_gec_Ga'
peft_model_id = '/home/jupyter/datasphere/project/rugec/notebooks/qwen_gec/ru_qwen7b_gec_Ga/checkpoint-261'

# Load adapter config
peft_config = PeftConfig.from_pretrained(peft_model_id)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=bnb_config,
    low_cpu_mem_usage=True,
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path, trust_remote_code=True)

# Merge LoRA adapter with base model
model = PeftModel.from_pretrained(base_model, peft_model_id)
model.config.use_cache = False

Loading checkpoint shards: 100%|██████████| 4/4 [03:30<00:00, 52.65s/it]


In [ ]:
# test = '/home/jupyter/datasphere/project/rugec/data/RULEC-GEC.test.tsv'
test = '/home/jupyter/datasphere/project/rugec/data/GERA.test.tsv'
df_test = pd.read_csv(test, delimiter='\t')

In [8]:
tokenizer

Qwen2TokenizerFast(name_or_path='Qwen/Qwen2-7B-Instruct', vocab_size=151643, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [5]:
from tqdm import tqdm

def generate_predictions(input_sentences, batch_size=64, max_new_tokens=512):
    all_predictions = []
    tokenizer.padding_side = "left"
    for i in tqdm(range(0, len(input_sentences), batch_size)):
        batch = input_sentences[i:i+batch_size]        

        prompts = [
            f"<|im_start|>system\nТы полезный ассистент для исправления грамматических ошибок.<|im_end|>\n<|im_start|>user\nИсправь ошибки в следующем предложении: {sentence}\nИсправленное предложение:<|im_end|>\n<|im_start|>assistant\n"
            for sentence in batch
        ]

        inputs = tokenizer(
            prompts,
            return_tensors="pt",
            # padding=True,
            padding='max_length',
            truncation=True,
            max_length=512,
        ).to(device)

        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_new_tokens=max_new_tokens,
                num_beams=5,
                early_stopping=True,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
            )

        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True, clean_up_tokenization_spaces=False)

        for full_output in decoded:
            if "Исправленное предложение:" in full_output:
                corrected = full_output.split("\n")[-1].strip()
            else:
                corrected = full_output.strip()
            all_predictions.append(corrected)

    return all_predictions



# Example usage
input_sentences = [
    "Он не был способен на найти решение.",
    "Это пример теста с ошибками.",
    "У меня был друг, который всегда говорил, что можно работать не напрягаясь и получать хорошие деньги, но мне казалось, что это просто шутка."
]

predictions = generate_predictions(input_sentences)
for sentence, prediction in zip(input_sentences, predictions):
    print(f"❌ Input: {sentence}")
    print(f"✅ Prediction: {prediction}\n")


100%|██████████| 1/1 [00:08<00:00,  8.20s/it]

❌ Input: Он не был способен на найти решение.
✅ Prediction: Он не был способен найти решение .

❌ Input: Это пример теста с ошибками.
✅ Prediction: Это пример теста с ошибками .

❌ Input: У меня был друг, который всегда говорил, что можно работать не напрягаясь и получать хорошие деньги, но мне казалось, что это просто шутка.
✅ Prediction: У меня был друг , который всегда говорил , что можно работать не напрягаясь и получать хорошие деньги , но мне казалось , что это просто шутка .



In [ ]:
df_test['corrected_sent'] = generate_predictions(df_test['corrupt_sent'])

100%|██████████| 21/21 [18:39<00:00, 53.30s/it]


In [ ]:
df_test

,corrupt_sent,correct_sent,corrected_sent
0,Как показана правда времени в романе А. С. Пуш...,Как показана правда времени в романе А. С. Пуш...,Как показана правда времени в романе А. С. Пуш...
1,"Действия в романе происходят в золотом веке , ...","Действия в романе происходят в золотом веке , ...","Действия в романе происходят в золотой век , в..."
2,Сам роман посвящен Пугачевщине .,Сам роман посвящен Пугачевщине .,Сам роман посвящен Пугачевщине .
3,Конечно были исторические произведения и до эт...,"Конечно , были исторические произведения и до ...","Конечно , были исторические произведения и до ..."
4,"К примеру самые ранние были трагедия "" Борис Г...","К примеру , самыми ранними были трагедия "" Бор...","К примеру , самые ранние были трагедия "" Борис..."
...,...,...,...
1309,"Тюрьма же обозначает заключение , узкие стены ...","Тюрьма же обозначает заключение , узкие стены ...","Тюрьма же обозначает заключение , узкие стены ..."
1310,Эти слова произносит сам Мцыри в своей исповед...,Эти слова произносит сам Мцыри в своей исповед...,Эти слова произносит сам Мцыри в своей исповед...
1311,"Сами слова , произнесённые мальчиком характери...","Сами слова , произнесённые мальчиком , характе...","Сами слова , произнесённые мальчиком , характе..."
1312,Образ Мцыри достаточно привлекателен его хариз...,Образ Мцыри достаточно привлекателен своей хар...,Образ Мцыри достаточно привлекателен его хариз...


In [ ]:
df_test = df_test.drop(columns=['correct_sent'])
df_test = df_test.rename(columns={'corrupt_sent' : 'corrupt', 'corrected_sent' : 'corrected'})

In [ ]:
df_test

,corrupt,corrected
0,Как показана правда времени в романе А. С. Пуш...,Как показана правда времени в романе А. С. Пуш...
1,"Действия в романе происходят в золотом веке , ...","Действия в романе происходят в золотой век , в..."
2,Сам роман посвящен Пугачевщине .,Сам роман посвящен Пугачевщине .
3,Конечно были исторические произведения и до эт...,"Конечно , были исторические произведения и до ..."
4,"К примеру самые ранние были трагедия "" Борис Г...","К примеру , самые ранние были трагедия "" Борис..."
...,...,...
1309,"Тюрьма же обозначает заключение , узкие стены ...","Тюрьма же обозначает заключение , узкие стены ..."
1310,Эти слова произносит сам Мцыри в своей исповед...,Эти слова произносит сам Мцыри в своей исповед...
1311,"Сами слова , произнесённые мальчиком характери...","Сами слова , произнесённые мальчиком , характе..."
1312,Образ Мцыри достаточно привлекателен его хариз...,Образ Мцыри достаточно привлекателен его хариз...


In [ ]:
df_test.to_csv('gera_qwen7b_v1.tsv', sep='\t', index=False)